In [1]:
import sys
if "google.colab" in sys.modules:
    !wget "https://raw.githubusercontent.com/IDAES/idaes-pse/main/scripts/colab_helper.py"
    import colab_helper
    colab_helper.install_idaes()
    colab_helper.install_ipopt()


import pyomo.environ as pyo
from pyomo.environ import *

--2025-08-30 13:40:00--  https://raw.githubusercontent.com/IDAES/idaes-pse/main/scripts/colab_helper.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5459 (5.3K) [text/plain]
Saving to: ‘colab_helper.py’

colab_helper.py     100%[===================>]   5.33K  --.-KB/s    in 0s      

2025-08-30 13:40:01 (47.7 MB/s) - ‘colab_helper.py’ saved [5459/5459]

Installing idaes via pip...
idaes was successfully installed
idaes, version 2.8.0


Running idaes get-extensions to install Ipopt, k_aug, and more...
Checking solver versions:
Ipopt 3.13.2 (x86_64-pc-linux-gnu), ASL(20190605)



[K_AUG] 0.1.0, Part of the IDAES PSE framework
Please visit https://idaes.org/ (x86_64-pc-linux-gnu), ASL(20190605)



Couenne 0.5.8 -- an Open-Source solver for Mixed Integer Nonl

In [4]:
rxns1=[ 'v1_1','v2_1' ,'v3_1','v4_1','v_bio1']
mets1=['m1_1','m2_1','m3_1','m4_1']
lb1=[0,0,0,0]
ub1=[1,1,1,1]
S1={('m1_1','v1_1'):-1,('m1_1','v2_1'):1 ,('m1_1','v3_1'):0,('m1_1','v4_1'):0,('m1_1','v5_1'):0,('m1_1','v_bio1'):0 ,\
    ('m2_1','v1_1'):0 ,('m2_1','v2_1'):0,('m2_1','v3_1'):-1,('m2_1','v4_1'):1,('m2_1','v5_1'):0,('m2_1','v_bio1'):0 ,\
    ('m3_1','v1_1'):0 ,('m3_1','v2_1'):-1,('m3_1','v3_1'):1,('m3_1','v4_1'):0 , ('m3_1','v5_1'):0 ,('m3_1','v_bio1'):-1,\
    ('m4_1','v1_1'):0 ,('m4_1','v2_1'):0,('m4_1','v3_1'):1,('m4_1','v4_1'):0 , ('m4_1','v5_1'):-1 ,('m4_1','v_bio1'):0
}
rxns2=['v1_2', 'v2_2','v3_2','v4_2','v_bio2']
mets2=['m1_2','m2_2','m3_2','m4_2']
lb2=[0,0,0,0,0,0]
ub2=[1,1,1,1,1,1]
S2={('m1_2','v1_2'):1 ,('m1_2','v2_2'):-1 ,('m1_2','v3_2'):0 ,('m1_2','v4_2'):0,('m1_2','v_bio2'):0 ,\
    ('m2_2','v1_2'):0 ,('m2_2','v2_2'):0 ,('m2_2','v3_2'):1 ,('m2_2','v4_2'):-1,('m2_2','v_bio2'):0 ,\
    ('m3_2','v1_2'):0 ,('m3_2','v2_2'):1 ,('m3_2','v3_2'):-1 ,('m3_2','v4_2'):0 ,('m3_2','v_bio2'):-1 ,\
    ('m4_2','v1_2'):0 ,('m4_2','v2_2'):0 ,('m4_2','v3_2'):1 ,('m4_2','v4_2'):0 ,('m4_2','v_bio2'):0
}
lb=[0,0,0,0,0,0,0,0,0,0]
ub=[1,1,1,1,1,1,1,1,1,1]

In [5]:
import pandas as pd
df = pd.Series(S2).unstack(fill_value=0)

print(df)

df = pd.Series(S1).unstack(fill_value=0)

print(df)

      v1_2  v2_2  v3_2  v4_2  v_bio2
m1_2     1    -1     0     0       0
m2_2     0     0     1    -1       0
m3_2     0     1    -1     0      -1
m4_2     0     0     1     0       0
      v1_1  v2_1  v3_1  v4_1  v5_1  v_bio1
m1_1    -1     1     0     0     0       0
m2_1     0     0    -1     1     0       0
m3_1     0    -1     1     0     0      -1
m4_1     0     0     1     0    -1       0


In [7]:
# create a model
model = ConcreteModel()

#create Sets
model.N = Set(initialize=rxns1 + rxns2)

model.M = Set(initialize=mets1 + mets2)


biomass_id=['v_bio1','v_bio2']

model.lb = pyo.Param(model.N, initialize={rxn: lb[i] for i, rxn in enumerate(rxns2 + rxns1)})
model.ub = pyo.Param(model.N,initialize={rxn: ub[i] for i, rxn in enumerate(rxns2 + rxns1)})

# Define the variable with the custom bounds function
model.v = pyo.Var(model.N)

model.lamda=pyo.Var(model.M)

model.eta_UB=pyo.Var(model.N ,domain=pyo.NonNegativeReals)

model.eta_Uptake=pyo.Var(model.N ,domain=pyo.NonNegativeReals)

model.eta_LB=pyo.Var(model.N,domain=pyo.NonNegativeReals)

model.biomass = Set(initialize=biomass_id, within=model.N)

model.obj=pyo.Objective(expr=sum(model.v[i] for i in model.biomass), sense=pyo.maximize)


def massbalance_rule(mdl, m):
    if m in mets1:
        return sum(S1[m, n] * mdl.v[n] for n in rxns1) == 0
    elif m in mets2:
        return sum(S2[m, n] * mdl.v[n] for n in rxns2) == 0
    else:
        # In case model.M contains something unexpected
        return pyo.Constraint.Skip

model.massbalance = pyo.Constraint(model.M, rule=massbalance_rule)

def UB_const_species_rule(mdl, r):
          return mdl.v[r] <= mdl.ub[r]
model.ub_species= pyo.Constraint(model.N, rule=UB_const_species_rule)

def LB_const_species_rule(mdl, r):
      return -1*mdl.v[r] <= -1*mdl.lb[r]
model.lb_species= pyo.Constraint(model.N, rule=LB_const_species_rule)

model.uptakespecices1=pyo.Constraint(expr=model.v['v4_1']<= model.v['v4_2'])


model.uptakespecices2_2=pyo.Constraint(expr=model.v['v1_2']<= model.v['v1_1'])

def dual_const_species1_rule(mdl, n):
   if n not in ['v_bio1'] and n in rxns1:
     if n in ['v4_1']:
      return sum(S1[m,n]*mdl.lamda[m] for m in mets1)+mdl.eta_Uptake[n]+ mdl.eta_UB[n]-mdl.eta_LB[n]==0

     return sum(S1[m,n]*mdl.lamda[m] for m in mets1)+ mdl.eta_UB[n]-mdl.eta_LB[n]==0

   if n in ['v_bio1'] :
       return sum(S1[m,n]*mdl.lamda[m] for m in mets1)+ mdl.eta_UB[n]-mdl.eta_LB[n]==1

   if n not in ['v_bio2'] and n in rxns2:
    if n in ['v1_2']:
       return sum(S2[m,n]*mdl.lamda[m] for m in mets2)+mdl.eta_Uptake[n]+ mdl.eta_UB[n]-mdl.eta_LB[n]==0

    return sum(S2[m,n]*mdl.lamda[m] for m in mets2)+ mdl.eta_UB[n]-mdl.eta_LB[n]==0

   if n in ['v_bio2'] :
       return sum(S2[m,n]*mdl.lamda[m] for m in mets2)+ mdl.eta_UB[n]-mdl.eta_LB[n]==1

model.dual_species= pyo.Constraint(model.N, rule=dual_const_species1_rule)


model.dual_eq_primal_species1=pyo.Constraint(expr=model.v['v4_2']*model.eta_Uptake['v4_1']+\
                                             sum(model.ub[n]*model.eta_UB[n] for n in rxns1)-\
                                             sum(model.lb[n]*model.eta_LB[n] for n in rxns1 )==model.v[biomass_id[0]])

model.dual_eq_primal_species2=pyo.Constraint(expr=model.v['v1_1']*model.eta_Uptake['v1_2']+\
                                             sum(model.ub[n]*model.eta_UB[n] for n in rxns2)-\
                                             sum(model.lb[n]*model.eta_LB[n] for n in rxns2 )==model.v[biomass_id[1]])


result=SolverFactory('ipopt').solve(model)

print('Species1 growth rate: ',model.v[biomass_id[0]].value,'\nSpecies2 growth rate: ',model.v[biomass_id[1]].value,'\nCommunity growth rate:', model.obj())
print("\nSolver Status:", result.solver.status)

Species1 growth rate:  6.802573988169454e-09 
Species2 growth rate:  1.6744048034053816e-09 
Community growth rate: 8.476978791574835e-09

Solver Status: ok
